#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [1]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [2]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [3]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [4]:
test_story = Story()
test_story.__str__()

"id='23cab160-6875-411d-9624-d2aed4da6183' genre='#toBeDefined' logline='#toBeDefined' summary='#toBeDefined' plotPoints=[] character_ids=[] season_ids=[]"

In [ ]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union, Literal

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> str:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is optional. You can pass 'None' and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story.__str__()
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result.__str__()

@tool
def update_story(updated_story: dict, runtime: ToolRuntime) -> Union[Literal["failure"], Literal["success"]]:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json. it should contain all the fields of a Story object.
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "success"
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "failure"
    

@tool
def write_to_database(updated_story: Story):
    """write the new updates so that they are persisted"""
    pass

@tool
def read_from_database():
    """read the latest state"""
    pass


# @tool
# def update_story(updated_story: dict[str, Any]):
#     """
#     Update the old version of the story if there is a new version
    
#     updated_story: this contains the updated version of the story which will replace the old version
    
#     """
#     print("update_story: called")
#     print("update_story: updated_story = {}".format(updated_story))
#     story_json = updated_story

    


##### Agent definition

In [14]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

# gpt_llm = ChatOllama(
#     model="gpt-oss:20b", 
#     reasoning="high",
#     verbose=True,
    
#     # temperature=0.5,
#     # disable_streaming=True
#     # format=Story.model_dump_json(), # type: ignore
# ).bind_tools([read_or_create_story, update_story])

my_agent = create_agent(
    "ollama:gpt-oss:20b",
    tools=[read_or_create_story, update_story],
    checkpointer=InMemorySaver(), # this retains the conversation history
    system_prompt="""
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """,
)

# my_agent = my_agent.bind_tools([update_story_variable])

# my_agent = create_agent("ollama:gpt-oss:20b", tools=[update_story_variable], response_format=Story)
# my_agent = init_chat_model("ollama:gpt-oss:20b",)


# my_agent = init_chat_model("openai:gpt-5.2") # testing cloud gpt 5.2
# my_agent.bind_tools([update_story_variable]) # binding for the (local) agent


##### Running the agent

In [28]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union

# def chatWithScenarist(newSession: bool = True, past_messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = []):

messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = [
        SystemMessage(
            """
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """.format(story_json)),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

            # Do not stop simply after calling a tool; tell me what you did and move forward.
            
            # Return complete story json when you update something.

            # Use your intelligence to provide a deep and or explorative answer for the given field.
            # Make your answers only so long as it is necessary.
            # You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative.

messages_for_agent: list[dict[str, str]] = []

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    # messages.append(HumanMessage(user_input))
    # messages_for_agent.append({
    #     "role": "user", "content": user_input
    # })


    # response = my_agent.invoke(
    #     {'messages': [{"role": "user", "content": user_input}]},
    #     {'configurable': {"thread_id": 1}}
    #     )
    # print(response)
    # ---
    response_text: Union[str, None] = None
    streaming_counter = 0
    # for chunk in my_agent.stream(input=messages):
    for stream_mode, chunk in my_agent.stream({
            'messages': [{"role": "user", "content": user_input}]},
            {'configurable': {"thread_id": 1}},
            stream_mode=['messages', 'tasks']
        ):
        # latest_message = chunk["messages"][-1]
        
        # latest_message = chunk
        # print("text: {} \ntype: {} \ncontent: {} \ncontent blocks: {} \nresponse metadata: {} \nat loop: {}".format(chunk.text, chunk.type, chunk.content, chunk.content_blocks, chunk.response_metadata, streaming_counter))
        # print("chunk: {}".format(chunk), end="", flush=True)
        
        # print('mode: {}'.format(stream_mode))
        # print("chunk: {}".format(chunk))
        if stream_mode == 'tasks':
            pass
        elif stream_mode == 'messages':            
            ai_message: AIMessageChunk = chunk[0]
            print(ai_message.content, end="", flush=True)

            if ai_message.content and ai_message.content_blocks[0]["type"] == "text":
                if response_text is None:
                    response_text = ai_message.content # type: ignore
                else:
                    response_text += ai_message.content # type: ignore
        
        
                
        streaming_counter += 1

    # response = AIMessage(content=response_text)
    
    # messages.append(response)
    
    display(Markdown(response_text))
    
    # latest_message = response['messages'][-1]
    # latest_message_content = latest_message.content
    # display(Markdown(latest_message_content))
    
    
    
# chatWithScenarist(False, messages) # type: ignore

user: I want to write a story
I’m ready to help you start the story, but first we need to set the **genre** (e.g., fantasy, thriller, coming‑of‑age, eco‑drama, horror, romance, etc.).  

Once you tell me the genre, I’ll lock it in and we can move on to the logline.

I’m ready to help you start the story, but first we need to set the **genre** (e.g., fantasy, thriller, coming‑of‑age, eco‑drama, horror, romance, etc.).  

Once you tell me the genre, I’ll lock it in and we can move on to the logline.

user: sitcom
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
success**Genre set:** Sitcom

We’re now ready to craft a logline that captures the humor and premise of a sitcom centered around a river.  
Do you have any ideas for the setting (e.g., a quirky town, a riverfront community center), main characters, or a central comedic conflict?  

Feel free to give me a sentence or two, and I’ll turn it into a concise logline and outline the first few beats.

success**Genre set:** Sitcom

We’re now ready to craft a logline that captures the humor and premise of a sitcom centered around a river.  
Do you have any ideas for the setting (e.g., a quirky town, a riverfront community center), main characters, or a central comedic conflict?  

Feel free to give me a sentence or two, and I’ll turn it into a concise logline and outline the first few beats.

user: did you write the river to the database?
Yes.  
The title **“River”** was saved in the database and the genre was updated to **Sitcom**.  
If you need anything else—logline, character sketches, or plot beats—just let me know.

Yes.  
The title **“River”** was saved in the database and the genre was updated to **Sitcom**.  
If you need anything else—logline, character sketches, or plot beats—just let me know.

user: yes give me some ideas
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
success**Logline (already stored)**  
*A down‑on‑their‑luck group of friends run a quirky riverfront community center, trying to keep the town’s beloved river flowing while juggling their own hilarious mishaps.*

---

### 1️⃣ Summary (draft for the story object)

> The town of Willowbend relies on the slow‑moving River Willow for its charm, culture, and the only public pool in town. A flailing city council decides to hand over the maintenance of the river and its surrounding park to a rag‑tag group of locals—each with their own eccentricities. They form the “River Rescue Squad,” a community center that hosts weekend yoga, improv nights, and a half‑hearted “River Cleanup” club. Chaos ensues when a mysterious corporate developer threatens to build a luxury marina, the center’s quirky founder’s ex‑spouse returns, and the river itself starts behaving like a diva 

success**Logline (already stored)**  
*A down‑on‑their‑luck group of friends run a quirky riverfront community center, trying to keep the town’s beloved river flowing while juggling their own hilarious mishaps.*

---

### 1️⃣ Summary (draft for the story object)

> The town of Willowbend relies on the slow‑moving River Willow for its charm, culture, and the only public pool in town. A flailing city council decides to hand over the maintenance of the river and its surrounding park to a rag‑tag group of locals—each with their own eccentricities. They form the “River Rescue Squad,” a community center that hosts weekend yoga, improv nights, and a half‑hearted “River Cleanup” club. Chaos ensues when a mysterious corporate developer threatens to build a luxury marina, the center’s quirky founder’s ex‑spouse returns, and the river itself starts behaving like a diva (think sudden whirlpools, talking reeds, and a mischievous koi). Every episode ends with a goofy town meeting that somehow saves the day, or at least leaves the mayor scratching his head.

---

### 2️⃣ Potential Main Characters

| Name | Role | Quirk / Backstory | Typical Comedy Hook |
|------|------|-------------------|---------------------|
| **Maya “Murmur” Delgado** | Former social worker turned center manager | Has an obsessive compulsion to keep the river “in tune” – uses a harmonica at every decision | Mistakes the river’s “mood” for human emotions, leading to over‑dramatic decisions |
| **Owen “Owl” Patel** | Owner of the fish market, now a “River Safety” consultant | Claims to talk to fish, insists the koi have a secret society | Tries to hold a fish council, misinterprets fish gossip |
| **Rae “Rhythm” Kim** | Yoga instructor and aspiring singer | Uses river sounds as her soundtrack, tries to “compose” the water | Tries to have the river sing a lullaby, causing a ripple effect |
| **Hank** | The town’s eccentric mayor | Believes the river is a sentient being that needs a personal trainer | Gives the river a “fitness plan” that includes “river jogging” |
| **Sam** | City council liaison, secretly a conspiracy theorist | Always reads the river as a cryptic message | Interprets a sudden current change as a clue to a hidden treasure |

---

### 3️⃣ Episode‑Level Plot Ideas (just a few seeds)

1. **Pilot** – The center opens, but the river refuses to let anyone use the splash pad. Maya tries a “river meditation” that accidentally starts a water‑based chant that attracts a swarm of flies.
2. **River vs. Marina** – The developer's bulldozer arrives; the crew attempts to build a “temporary dam” to protect the river, but ends up creating an impromptu floating dance studio.
3. **Koi Conspiracy** – Owen holds a “koi council” meeting; the fish apparently “vote” on the center’s weekly menu, leading to a fish‑approved pizza topping.
4. **The River’s Secret** – The center discovers an old logbook in the riverbank. The entries are cryptic and seem to hint at a treasure buried beneath the riverbed, causing the crew to search while the mayor’s fish market collapses.

---

### 4️⃣ Potential Season Arc Ideas

| Season | Hook | Major Conflict | Resolution |
|--------|------|----------------|------------|
| 1 | Opening the center and establishing quirky dynamics | Corporate developer threatens to close the center | Community rallies; the developer is turned into a volunteer |
| 2 | The “River’s” growing personality (talking reeds, singing koi) | Mayor tries to commercialize the river; the river “sings” to the public to stop it | River gets a “voice” in town council, the mayor must listen |
| 3 | Introducing a rival town with a “perfect” river | The rival town tries to lure the community center staff away | Staff returns, learns to embrace imperfection |

---

#### Critique & Inspiration Notes

- **Balancing Exposition and Humor:** Sitcoms thrive on quick, punchy jokes. Make sure each character’s quirk gets a chance to surface early (e.g., Maya’s harmonica at the opening meeting).
- **Visual Gags with the River:** Consider physical comedy—water splashing at the wrong moment, fish popping out of unexpected places, a sudden whirlpool in the center’s pool that turns a serious talk show into a chaotic splash.
- **Social Commentary (light touch):** The struggle between development and community preservation is a common trope—think *Parks & Recreation* meets *The Office*’s small‑town vibe.
- **Recurring Set‑Pieces:** The “River’s Mood Board” (a chalkboard where characters write the river’s feelings), a weekly “River Bingo” card that everyone fills, and a “Koi Karaoke” night.

Let me know which elements resonate, or if you’d like deeper outlines for a particular episode, character arc, or season plan!